# Web Scraping completo

Esta es la estructura de un documento típico HTML

![estructura](estructura.png)

El objetivo será extraer la data de la siguiente página web: http://books.toscrape.com/

Utilizando el paquete 'requests' vamos a obtener el html de la página.

In [22]:
main_url = "http://books.toscrape.com/index.html"

import requests
result = requests.get(main_url)

result.text[:1000]

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

El resultado es un poco desordenado, vamos a utilizar el paquete 'BeautifulSoup' para verlo un poco mejor.

In [23]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(result.text, 'html.parser')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

Definamos una función para solicitar y analizar el html.

In [24]:
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

## Encontremos las URLs en la página

La estructura del código html es la siguiente:

![inspeccionar](inspeccionar.png)

Con el siguiente código podemos encontrar la primera ocurrencia de la etiqueta.

In [25]:
soup.find("article", class_ = "product_pod")

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

Bien, pero vamos un nivel más profundo.

In [26]:
soup.find("article", class_ = "product_pod").div.a

<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>

'href' permite extraer solo el enlace.

In [27]:
soup.find("article", class_ = "product_pod").div.a.get('href')

'catalogue/a-light-in-the-attic_1000/index.html'

Logramos extraer la URL del primer producto, ahora recopilemos las URLs de todos los productos.

In [28]:
main_page_products_urls = [x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

print(str(len(main_page_products_urls)) + " URLs recopiladas")
print("Un ejemplo:")
main_page_products_urls[0]

20 URLs recopiladas
Un ejemplo:


'catalogue/a-light-in-the-attic_1000/index.html'

Estas URLs corresponden al 'path' absoluto de los libros; para tener la URL completa, basta con agregar la URL de la página principal:  http://books.toscrape.com/index.html

In [29]:
def getBooksURLs(url):
    soup = getAndParseURL(url)
    # Remover la parte que dice 'index.html' del enlace
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")])

## Encontrar las URLs de las categorías de los libros

![categoria](categoria.png)

Podemos observar que las categorías siguen el patrón 'catalogue/category/books'; utilicemos el paquere de expresiones regulares.

In [30]:
import re

categories_urls = [main_url + x.get('href') for x in soup.find_all("a", href=re.compile("catalogue/category/books"))]
categories_urls = categories_urls[1:] # Removemos la primera que corresponde a todos los libros

print(str(len(categories_urls)) + " URLs recopiladas")
print("Algunos ejemplos:")
categories_urls[:5]

50 URLs recopiladas
Algunos ejemplos:


['http://books.toscrape.com/index.htmlcatalogue/category/books/travel_2/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/mystery_3/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/historical-fiction_4/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/sequential-art_5/index.html',
 'http://books.toscrape.com/index.htmlcatalogue/category/books/classics_6/index.html']

## Scrapear toda la data

Nótese que no todos los libros se muestran en una página, hay 50 de estas, y hay que darle al botón de siguiente para ver todos los libros.

![paginas](paginas.png)

### Extraer todas las URLs de las páginas

![pagina](pagina-estructura.png)

El botón 'next' contiene el patrón 'page'. Hay que tener cuidado porque el botón 'previous' también lo contiene. En ese sentido, se debe tomar el segundo resultado, correspondiente a la página siguiente.

In [32]:
# Guardar todos los resultados en una lista
pages_urls = [main_url]

soup = getAndParseURL(pages_urls[0])

# El siguiente while evalúa que existan 2 emparejamientos,
# si solo existe uno, es porque estamos en la primera o última página
# El while se detiene cuando se llegue a la última página

while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    
    # Nueva URL
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    
    # Agregar la URL a la lista
    pages_urls.append(new_url)
    
    # capturtar la siguiente página
    soup = getAndParseURL(new_url)
    
print(str(len(pages_urls)) + " URLs capturadas")
print("Algunos ejemplos:")
pages_urls[:5]



Wall time: 0 ns
50 URLs capturadas
Algunos ejemplos:


['http://books.toscrape.com/index.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

Nótese que las URLs poseen un patrón al final, podríamos haber hecho un contador que llegue hasta 50, pero esto dejaría de funcionar si el número de páginas cambia.

Una solución es incrementar el valor hasta encontrar un error en la página.

![404](404.png)

In [33]:
result = requests.get("http://books.toscrape.com/catalogue/page-50.html")
print("Estado de la página 50: " + str(result.status_code))

result = requests.get("http://books.toscrape.com/catalogue/page-51.html")
print("Estado de la página 51: " + str(result.status_code))

Estado de la página 50: 200
Estado de la página 51: 404


Podemos utilizar esto para obtener todas las URLs.

In [34]:
pages_urls = []

new_page = "http://books.toscrape.com/catalogue/page-1.html"
while requests.get(new_page).status_code == 200:
    pages_urls.append(new_page)
    new_page = pages_urls[-1].split("-")[0] + "-" + str(int(pages_urls[-1].split("-")[1].split(".")[0]) + 1) + ".html"
    

print(str(len(pages_urls)) + " URLs capturadas")
print("Algunos ejemplos:")
pages_urls[:5]

50 URLs capturadas
Algunos ejemplos:


['http://books.toscrape.com/catalogue/page-1.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

### Obtener todas las URLs de los productos

In [35]:
booksURLs = []
for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " URLs capturadas")
print("Algunos ejemplos:")
booksURLs[:5]

1000 URLs capturadas
Algunos ejemplos:


['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html']

1000 es el número de libros que se encuentran en el sitio web.

### Extraer los datos de cada libro

![data](data.png)

In [36]:
names = []
prices = []
nb_in_stock = []
img_urls = []
categories = []
ratings = []

# Scrapear la data de la URL de cada libro
for url in booksURLs:
    soup = getAndParseURL(url)
    # Nombre
    names.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    # Precio
    prices.append(soup.find("p", class_ = "price_color").text[2:]) # Quitar el símbolo de dinero
    # Número de unidades disponibles
    nb_in_stock.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # Quitar caracteres no numéricos
    # URL de la imagen
    img_urls.append(url.replace("index.html", "") + soup.find("img").get("src"))
    # Categoría
    categories.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])
    # Calificación / rating
    ratings.append(soup.find("p", class_ = re.compile("star-rating")).get("class")[1])
    
# Agregar los datos en un dataframe de pandas
import pandas as pd

scraped_data = pd.DataFrame({'nombre': names, 'precio': prices, 'No. disponibles': nb_in_stock, "URL img": img_urls, "Categoría": categories, "rating": ratings})
scraped_data.head()

,nombre,precio,No. disponibles,URL img,Categoría,rating
0,A Light in the Attic,51.77,22,http://books.toscrape.com/catalogue/a-light-in...,poetry_23,Three
1,Tipping the Velvet,53.74,20,http://books.toscrape.com/catalogue/tipping-th...,historical-fiction_4,One
2,Soumission,50.10,20,http://books.toscrape.com/catalogue/soumission...,fiction_10,One
3,Sharp Objects,47.82,20,http://books.toscrape.com/catalogue/sharp-obje...,mystery_3,Four
4,Sapiens: A Brief History of Humankind,54.23,20,http://books.toscrape.com/catalogue/sapiens-a-...,history_32,Five
